In [1]:
!pip3 install networkx

In [2]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as seabornInstance
from sqlalchemy import Column, Integer, String, Float, DateTime, Boolean, func
from iotfunctions import base
from iotfunctions import bif
from iotfunctions.metadata import EntityType
from iotfunctions.db import Database
from iotfunctions.enginelog import EngineLogging
from iotfunctions import estimator
import datetime as dt
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
%matplotlib inline
import pprint as pprint
import json
import networkx as nx

In [3]:
with open('credentials_as_monitor_demo.json', encoding='utf-8') as F:
    credentials = json.loads(F.read())
    
db_schema=None
db = Database(credentials=credentials)
print (db)

In [4]:
# load all entity types - similar to the explore tab on the dashboard
metadata = db.http_request(object_type='allEntityTypes', object_name='', request='GET', payload={},  object_name_2='')

In [5]:
pp = pprint.PrettyPrinter(depth = 3)
metadataJson = json.loads(metadata)
pp.pprint(metadataJson)

[{'archivedDate': None,
  'createdBy': None,
  'createdTimestamp': '2021-01-05T16:09:07.809+0000',
  'dataItemDto': [{...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
 

                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},


  'entityTypeId': 8791,
  'logicalInterfaceId': None,
  'metricTableName': 'iot_ekstrom_thingy',
  'metricTimestampColumn': 'RCV_TIMESTAMP_UTC',
  'name': 'ekstrom_thingy',
  'origin': 'IOTP',
  'schemaName': 'BLUADMIN',
  'status': 'ACTIVE',
  'updatedBy': None,
  'updatedTimestamp': '2020-11-27T11:02:46.176+0000'},
 {'archivedDate': None,
  'createdBy': None,
  'createdTimestamp': '2020-11-11T14:29:33.039+0000',
  'dataItemDto': [{...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
       

                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...}],
  'description': None,
  'dimensionTableName': 'iot_lm_adeunis_ctg',
  'entityTypeId': 13002,
  'logicalInterfaceId': None,
  'metricTableName': 'iot_lm_adeunis',
  'metricTimestampColumn': 'RCV_TIMESTAMP_UTC',
  'name': 'lm_adeunis',
  'origin': 'IOTP',
  'schemaName': 'BLUADMIN',
  'status': 'ACTIVE',
  'updatedBy': None,
  'updatedTimestamp': '2021-02-16T13:17:03.609+0000'},
 {'archivedDate': None,
  'createdBy': None,
  'createdTimestamp': '2020-12-09T10:55:33.177+0000',
  'dataItemDto': [{...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {

  'createdTimestamp': '2021-01-25T18:10:27.994+0000',
  'dataItemDto': [{...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...}],
  'description': None,
  'dimensionTableName': 'iot_chesebrough_thingy_ctg',
  'entityTypeId': 13214,
  'logicalInterfaceId': None,
  'metricTableName': 'iot_chesebrough_thingy',
  'metricTimestampColumn': 'RCV_TIMESTAMP_UTC',
  'name': 'Chesebrough_Thingy',
  'origin': 'IOTP',
  'schemaName': 'BLUADMIN',
  'status': 'ACTIVE',
  'updatedBy': None,
  'updatedTimestamp': '2021-01-25T18:10:27.994+0000'},
 {'archivedDate': None,
  'createdBy': None,
  'createdTimestamp': '2020-12-11T14:46:07.999+0000',
  'dataItemDto': [{...}, {...}, {...}, {...}],
  'description': '\n    Base class for custom entity types\n    ',
  'dimensionTableName': 'io

                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...}],
  'description': None,
  'dimensionTableName': 'iot_rhlines01_ctg',
  'entityTypeId': 10070,
  'logicalInterfaceId': None,
  'metricTableName': 'iot_rhlines01',
  'metricTimestampColumn': 'RCV_TIMESTAMP_UTC',
  'name': 'RHLines01',
  'origin': 'IOTP',
  'schemaName': 'BLUADMIN',
  'status': 'ACTIVE',
  'updatedBy': None,
  'updatedTimestamp': '2020-12-14T07:47:48.510+0000'},
 {'archivedDate': None,
  'createdBy': None,
  'createdTimestamp': '2021-01-27T09:15:07.041+0000',
  'dataItemDto': [{...}, {...}],
  'description': 'MVI-device',
  'dimensionTableName': 'mvi-device_dimension',
  'entityTypeId': 13337,
  'logicalInterfaceId': None,
  'metricTableName': 'mvi-device',
  'metricTimestampColumn': 'evt_timestamp',
  'name': 'MVI-device',
  'origin': 'AS_SAMPLE',
  'schemaName': 'BLUADMIN',
  'status': 'ACTIVE',
  'updatedBy': None,
  'upd

                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...}],
  'description': None,
  'dimensionTableName': 'iot_ifc_demo_hvac_ctg',
  'entityTypeId': 14133,
  'logicalInterfaceId': None,
  'metricTableName': 'iot_ifc_demo_hvac',
  'metricTimestampColumn': 'RCV_TIMESTAMP_UTC',
  'name': 'IFC_DEMO_HVAC',
  'origin': 'IOTP',
  'schemaName': 'BLUADMIN',
  'status': 'ACTIVE',
  'updatedBy': None,
  'updatedTimestamp': '2021-02-16T13:30:34.869+0000'},
 {'archivedDate': None,
  'createdBy': None,
  'createdTimestamp': '2021-02-19T12:03:32.853+0000',
  'dataItemDto': [{...}, {...}, {...}, {...}, {...}, {...}, {...}],
  'description': '\n    Base class for custom entity types\n    ',
  'dimensionTableName': 'test_demo_102_dimension',
  'entityTypeId': 14929,
  'logicalInterfaceId': 

                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...}],
  'description': None,
  'dimensionTableName': 'iot_demodevice_2_ctg',
  'entityTypeId': 1219,
  'logicalInterfaceId': None,
  'metricTableName': 'iot_demodevice_2',
  'metricTimestampColumn': 'RCV_TIMESTAMP_UTC',
  'name': 'demodevice_2',
  'origin': 'IOTP',
  'schemaName': 'IOTANALYTICS',
  'status': 'ACTIVE',
  'updatedBy': None,
  'updatedTimestamp': '2020-10-07T06:33:05.337+0000'},
 {'archivedDate': None,
  'createdBy': None,
  'createdTimestamp': '2021-02-19T08:23:43.718+0000',
  'dataItemDto': [{...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
          

  'name': 'TRMJQTThing',
  'origin': 'IOTP',
  'schemaName': 'BLUADMIN',
  'status': 'ACTIVE',
  'updatedBy': None,
  'updatedTimestamp': '2021-02-20T02:06:13.107+0000'},
 {'archivedDate': None,
  'createdBy': None,
  'createdTimestamp': '2021-02-24T07:11:16.684+0000',
  'dataItemDto': [{...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
     

  'createdTimestamp': '2020-09-24T16:18:01.946+0000',
  'dataItemDto': [{...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...},
                  {...}],
  'description': 'Equipment Turbines',
  'dimensionTableName': 'lloop_dimension',
  'entityType

In [6]:
# get my entity type
et = db.http_request(object_type='entityType', object_name='Z_Robot_Function', request='GET', payload={},  object_name_2='')

In [7]:
# explore my data
etJson = json.loads(et)
pp.pprint(etJson)

{'ex': 'Entity type not found for given name Z_Robot_Function',
 'url': 'https://api-beta.connectedproducts.internetofthings.ibmcloud.com/api/meta/v1/Monitor-Demo/entityType/Z_Robot_Function'}


In [8]:
engineInput = db.http_request(object_type='engineInput', object_name='z_robot_function', request='GET', payload={},  object_name_2='')

In [9]:
engineInputJson = json.loads(engineInput)
pp._depth = 5
pp.pprint(engineInputJson)

{'ex': "Entity type with name 'z_robot_function' not found",
 'url': 'https://api-beta.connectedproducts.internetofthings.ibmcloud.com/api/kpi/v1/Monitor-Demo/entityType/z_robot_function/engineInput'}


In [10]:
dataItems = engineInputJson['dataItems']

G=nx.DiGraph()

# get rid of unneeded stuff an fill dictionary
for el in dataItems:
    if el['type'] not in ('METRIC', 'DERIVED_METRIC'):
        continue
    
    G.add_node(el.get('name'))
    kpiFctDto = el['kpiFunctionDto']
    if kpiFctDto != None and not isinstance(kpiFctDto, str):
        kpiInput = kpiFctDto['input']
        for kpiInputEl in kpiInput:
            attr = kpiInput.get(kpiInputEl)
            #print (kpiInputEl, attr)
            if isinstance(attr, str):
                for el2 in dataItems:
                    if (el2.get('name') == attr):
                        G.add_edge(el2.get('name'), el.get('name'))
            
# remove isolated nodes
G.remove_nodes_from(list(nx.isolates(G)))

#pp.pprint(engineInputJson['dataItems'])
plt.figure(figsize=(7,5))
pos = nx.spring_layout(G)
nx.draw(G, with_labels = True, font_size=14)
#for p in pos:  # raise text positions
#    pos[p][1] += 0.07
#nx.draw_networkx_labels(G, pos)
plt.show()

KeyError: 'dataItems'